## Assignment 1 Erik Hystad

Requirements:
Nltk
    * Corpus.Brown
    * Corpus.Wordnet
    * Collocations
Jupyter Notebook



In [ ]:
import math
import nltk
from nltk.corpus import brown
from nltk.collocations import *
from nltk.corpus import wordnet
from tqdm import tqdm

 ### Task 1.1
First I will load the corpus and find all collocations with the BigramCollocationFinder.

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
corpus = brown.words()

finder = BigramCollocationFinder.from_words(corpus)
collocations = finder.nbest(bigram_measures.pmi, 10000)

#### Task 1.1.1
Then I will apply a frequency filter to the finder, which I here set to 6, to remove any collocations that
appear less than 6 times. I chose 6 to reduce somewhat the time the hypothesis testing in 1.1.2 would take to run.
Then I only keep collocations that are made up of nouns and adjectives.

After that I write these to a file, '1.1.1.txt'.

In [ ]:
finder.apply_freq_filter(10)
frequency_collocations = finder.nbest(bigram_measures.pmi, 10000)


tagged = [nltk.pos_tag(bigram) for bigram in frequency_collocations]
including = ['NN', 'JJ']


# A function to see if a word is nouns or adjectives, will include NNS etc...
def check(bigram):
    for _, cl in bigram:
        result = False
        for cls in including:
            if cls in cl:
                result = True
        if not result:
            return result
    return True


noun_and_adjectives_collocations = [bigram for bigram in tagged if check(bigram)]

with open("1_1_1.txt", 'w') as file:
    for bigram in noun_and_adjectives_collocations:
        file.write(bigram[0][0] + ' ' + bigram[1][0] + '\n')

#### Task 1.1.2

Here I did hypothesis testing on the remaining bigrams, the ones already filtered by nouns, adjectives and more than
10 occurrences.


In [ ]:
n = len(corpus)


def hypothesis_test(collocation, confidence=2.576):
    first, second = collocation
    sample_mean = (finder.ngram_fd[collocation] / n)
    mean_of_the_dist = ((corpus.count(first) / n) * (corpus.count(second) / n))
    t = (sample_mean - mean_of_the_dist) / (math.sqrt((sample_mean * (1 - sample_mean))**2/n))
    return t > confidence

with open("1_1_2.txt", 'w') as file:
    ts = []
    pbar = tqdm(total=len(noun_and_adjectives_collocations), desc='Hypothesis test')
    for bigram in noun_and_adjectives_collocations:
        if hypothesis_test((bigram[0][0], bigram[1][0])):
            file.write(bigram[0][0] + ' ' + bigram[1][0] + '\n')
        pbar.update()
    pbar.close()

#### 1.2



In [ ]:
with open('1_1_2.txt', 'r') as file:
    raw = file.read()

learned_collocations = [(x.split()[0].lower(), x.split()[1].lower()) for x in raw.split('\n') if x != '']
learned_collocations_firsts = set([x[0] for x in learned_collocations])
learned_collocations_seconds = set([x[1] for x in learned_collocations])


def correction_tool(first, second):
    for synset in wordnet.synsets(first):
        for synonym in synset.lemmas():
            f_synonym = synonym.name()
            if f_synonym in learned_collocations_firsts:
                second_learned = [x[1] for x in learned_collocations if x[0] == f_synonym]
                for s in second_learned:
                    for second_synset in wordnet.synsets(second):
                        for second_synonym in second_synset.lemmas():
                            s_synonym = second_synonym.name()
                            if s == s_synonym and first + ' ' + second != f_synonym + ' ' + s:
                                print('Changed', first + ' ' + second, 'to', f_synonym + ' ' + s, '!')
                                return f_synonym + ' ' + s
    print("Found no matching collocation.")

first, second = input("Write bigram to see if can be corrected to a collocation.")
correction_tool(first, second)